# MODELLI IR - Spazio vettoriale

Librerie -->nltk e scikit-learn

## CountVectorizer
**CouterVectorizer** classe per convertire il testo in una matrice di token. Usa la tf per la pesatura dei token e 
produce un modello con matrici sparse di tipo Numpy. Usa il modello **bag of word**.
Di default fa già 
- una tokenizzazione 
- la creazione del vocabolario
- costruzione della matrice

Il costruttore se usato senza argomenti usa le sue impostazioni di default che pero è possibile modificare passando funzioni custom.
Fa test pre-processing ma non rimuove le stopword.

In [151]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd


vectorizer=CountVectorizer()

#documenti da rappresentare nello spazio vettoriale
#ogni riga è un documento
corpus=["Racing games",
        "This document describes racing cars",
        "This document is about video games in general",
        "This is a nice racing video game"]

Per creare il modello usiamo:

In [152]:
mod_vect=vectorizer.fit_transform(corpus)

df = pd.DataFrame(mod_vect.toarray(), columns=vectorizer.get_feature_names_out())
print(df)


   about  cars  describes  document  game  games  general  in  is  nice  \
0      0     0          0         0     0      1        0   0   0     0   
1      0     1          1         1     0      0        0   0   0     0   
2      1     0          0         1     0      1        1   1   1     0   
3      0     0          0         0     1      0        0   0   1     1   

   racing  this  video  
0       1     0      0  
1       1     1      0  
2       0     1      1  
3       1     1      1  


che serve per creare il mdello dai documenti, quindi impara il vocabolario e lo trasforma in una matrice termini-documenti.

**Valori di ritorno** --> matrice sparsa Scipy, dove le riche sono i documenti e le colonne gli index termo per rappresentare i documenti nello spazio vettoriale. Nelle intersezioni si ha la tf, cioè la frequenza con cui una parola appare in quel documento.

**Parametri** --> lista dei documenti, iterabile

Per apprendere dai dati solo le statistiche per creare il modello senza apprendere il modello, quindi senza effettuare una trasformazione dei dati trasformandoli in una forma matriciale, ma calcolare solo il vocabolario usiamo

In [153]:
stats=vectorizer.fit(corpus)
print(stats) 
print(stats.get_feature_names_out())


CountVectorizer()
['about' 'cars' 'describes' 'document' 'game' 'games' 'general' 'in' 'is'
 'nice' 'racing' 'this' 'video']


**Valori di ritorno** --> un oggetto CountVectorizer, ma con le informazioni per creare il modello

**Parametri** --> lista dei documenti, iterabile

Poi usando questo nuovo CountVectorizer possiamo creare il modello usando

In [154]:
mtd=stats.transform(corpus)
df = pd.DataFrame(mtd.toarray(), columns=vectorizer.get_feature_names_out())
print(df)

   about  cars  describes  document  game  games  general  in  is  nice  \
0      0     0          0         0     0      1        0   0   0     0   
1      0     1          1         1     0      0        0   0   0     0   
2      1     0          0         1     0      1        1   1   1     0   
3      0     0          0         0     1      0        0   0   1     1   

   racing  this  video  
0       1     0      0  
1       1     1      0  
2       0     1      1  
3       1     1      1  


questa funzione è usata anche per poter trasformare i nuovi dati da inserire nel modello. In questo caso il modello non viene ricreato, ma il nuovo documento che viene inserito viene trasposto nello spazio vettoriale già esistente considerando i termini presenti nello spazio. termini che non sono mai stati visti non vengono inseriti, ma scartati.

## Cosine Similarity
è una funzione che calcola il coseno fra due vettori. In questo caso il coseno rappresenta la similarità fra due documenti.
1. se passiamo alla funzione solo la matrice del modello calcola la cos_sim fra i documenti usati per addrestrare il modello;
2. se passiamo alla funzione la matrice del modello e la query, la cos_sim è calcolata fra la query e ogni singolo documeto. In questo caso la query deve essere prima inserita nello spazio vettoriale con la  *fit_trasform()*.

In [155]:
# Calcolare la similarità coseno tra tutti i documenti
cos = cosine_similarity(mod_vect)
print("Similarità Coseno tra i documenti:")
print(cos)
print()

# Aggiungere una query al modello
query = vectorizer.transform(["racing game"])

# Calcolare la similarità coseno tra la query e i documenti
cos = cosine_similarity(query, mod_vect)
print("Similarità Coseno tra la query e i documenti:")
print(cos)

Similarità Coseno tra i documenti:
[[1.         0.31622777 0.25       0.28867513]
 [0.31622777 1.         0.31622777 0.36514837]
 [0.25       0.31622777 1.         0.4330127 ]
 [0.28867513 0.36514837 0.4330127  1.        ]]

Similarità Coseno tra la query e i documenti:
[[0.5        0.31622777 0.         0.57735027]]


# MODELLI IR - Tf-idf

## TfidfVectorizer
TfidfVectorizer è una classe che usa come metrica la tf-idf, funziona come il caso precedente.

# MODELLI IR - Inverted Index

Librerie --> gensim

Si potrebbe fare anche con scikit-learn usando **HashingVectorizer** ma usa come metrica la tf. Con gensim usando **Corpora.Dictionary** mappiamo ogni parola del testo ad un ID univoco, quindi usando un modello BoW

In [156]:
from gensim import corpora
from gensim import models
from gensim import similarities
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import snowball
import re

def my_tokenizer(text):
    """tokenization function"""
    sw=stopwords.words('english')
    stemmer=snowball.SnowballStemmer(language="english")
    tokens=word_tokenize(text)
    pruned=[stemmer.stem(t.lower()) for t in tokens \
            if re.search(r"^\w",t) and not t.lower() in sw]
    return pruned

documents=["This document describes racing cars",
        "This document is about video games in general",
        "This is a nice racing video game"]

texts=[my_tokenizer(d) for d in documents]

dictionary = corpora.Dictionary(texts)
print(dictionary)

# doc2bow per convertire i docs in BoW 
bow_corpus=[dictionary.doc2bow(text) for text in texts]
tfidf = models.TfidfModel(bow_corpus)

Dictionary<8 unique tokens: ['car', 'describ', 'document', 'race', 'game']...>


## Similarities.SpaseMatrixSimilarity
La funzione **`SparseMatrixSimilarity`** è una classe di Gensim utilizzata per calcolare la similarità tra una query e un insieme di documenti, rappresentati come matrici sparse (in formato **TF-IDF** o **Bag-of-Words**).

La funzione **`SparseMatrixSimilarity`** accetta i seguenti parametri:

### 1. **`corpus`** 
- **Descrizione**: La matrice sparsa che rappresenta il corpus di documenti.
- **Tipo**: Una matrice sparsa, generalmente una rappresentazione **TF-IDF** o **Bag-of-Words**.
- **Esempio**: `tfidf[bow_corpus]`
- **Funzione**: Il corpus è una matrice che contiene la rappresentazione numerica di tutti i documenti, dove le righe sono i documenti e le colonne sono i termini del vocabolario. Ogni valore nella matrice rappresenta l'importanza di un termine in un dato documento.

### 2. **`num_features`** 
- **Descrizione**: Il numero di **caratteristiche** (o dimensioni) nel vocabolario.
- **Tipo**: Un intero che rappresenta la lunghezza del vocabolario.
- **Esempio**: `len(dictionary)` (dove `dictionary` è il vocabolario che contiene tutte le parole uniche nei documenti).
- **Funzione**: Specifica la dimensione del vocabolario, ossia il numero di parole uniche presenti nel tuo corpus. Questo parametro è necessario per indicare la dimensione della matrice sparsa.

### 3. **`num_best`** 
- **Descrizione**: Il numero di documenti più simili da restituire.
- **Tipo**: Un intero (predefinito è 10).
- **Esempio**: `num_best=5`
- **Funzione**: Limita il numero di risultati restituiti, mostrando solo i documenti più simili alla query. Se non specificato, restituirà tutti i documenti con le loro similarità coseno rispetto alla query.

### 4. **`threshold`** 
- **Descrizione**: Una soglia di similarità minima. Se la similarità coseno tra un documento e la query è inferiore a questa soglia, il documento verrà escluso dai risultati.
- **Tipo**: Un valore float tra 0 e 1 (predefinito è 0.0).
- **Esempio**: `threshold=0.5`
- **Funzione**: Filtra i documenti con una similarità inferiore alla soglia specificata. Se impostato a 0.5, solo i documenti con una similarità maggiore o uguale a 0.5 saranno restituiti.

---

La funzione **`SparseMatrixSimilarity`** restituisce un oggetto che può essere utilizzato per calcolare la similarità coseno tra una query e il corpus di documenti.

### Ritorno:
- **Tipo**: Un oggetto di tipo **`SparseMatrixSimilarity`**.
- **Funzione**: L'oggetto restituito è utilizzato per calcolare la similarità tra un dato documento e il corpus di documenti. Quando si fornisce una query a questo oggetto, restituirà un array di similarità coseno tra la query e ogni documento nel corpus.


In [157]:

index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus],len(dictionary))
print(list(index))

#tokenizzazione della query
query_document = my_tokenizer("racing games")
query_bow = dictionary.doc2bow(query_document)

sims = index[tfidf[query_bow]]
print(list(enumerate(sims)))


[array([1.0000001 , 0.07613309, 0.07613309], dtype=float32), array([0.07613309, 1.        , 0.19339646], dtype=float32), array([0.07613309, 0.19339646, 1.        ], dtype=float32)]
[(0, 0.17312077), (1, 0.21988432), (2, 0.43976864)]


# MODELLI IR AVANZATI - N-grammi

Per utilizzare gli **n-grammi** con il **`TfidfVectorizer`**, basta impostare il parametro `ngram_range` durante la creazione dell'oggetto. Questo parametro definisce la lunghezza degli n-grammi da considerare.

```python
ngram_range=(min_n, max_n)
``` 
- min_n: la lunghezza minima degli n-grammi (incluso).
- max_n: la lunghezza massima degli n-grammi (incluso).

In [158]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import snowball
import re

def my_tokenizer(text):
        sw = stopwords.words('english')  
        stemmer = snowball.SnowballStemmer(language="english")  
        tokens = word_tokenize(text)
        pruned = [stemmer.stem(t) for t in tokens if re.search(r"^[a-zA-Z]", t) and not t in sw]
        return pruned

# Inizializza il vettorizzatore considerando i bigrammi
vectorizer = TfidfVectorizer(tokenizer=my_tokenizer, ngram_range=(1, 2))

corpus = ["This document describes racing cars",
          "This document is about videos of table games",
          "This is a nice racing video game",
          "Video killed the radio star"]

# Allena il modello sul corpus
model = vectorizer.fit_transform(corpus)
print("Caratteristiche (ngrammi) del vocabolario:")
print(vectorizer.get_feature_names_out())

# Esegui la trasformazione della query nel modello
query = vectorizer.transform(["video game"])
cos = cosine_similarity(query, model)
print("Similarità coseno tra la query e i documenti:")
print(cos)

Caratteristiche (ngrammi) del vocabolario:
['car' 'describ' 'describ race' 'document' 'document describ'
 'document video' 'game' 'kill' 'kill radio' 'nice' 'nice race' 'race'
 'race car' 'race video' 'radio' 'radio star' 'star' 'tabl' 'tabl game'
 'video' 'video game' 'video kill' 'video tabl']
Similarità coseno tra la query e i documenti:
[[0.         0.30389824 0.59923094 0.11299246]]


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


# MODELLI AVANZATI IR - LSI

In [159]:
from gensim import corpora
from gensim import models
from gensim import similarities
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import snowball
import re

def my_tokenizer(text):
    sw=stopwords.words('english')
    stemmer=snowball.SnowballStemmer(language="english")
    tokens=word_tokenize(text)
    pruned=[stemmer.stem(t.lower()) for t in tokens \
            if re.search(r"^[a-zA-Z]",t) and not t.lower() in sw]
    return pruned

documents=["Indian government goes for open source software",
"Debian 3.0 Woody released",
"Wine 2.0 released with fixes for Gentoo 1.4 and Debian 3.0",
"gnuPOD released: iPOD on Linux… with GPLed software",
"Gentoo servers running at open source mySQL database",
"Dolly the sheep not totally identical clone",
"DNA news: introduced low-cost human genome DNA chip",
"Malaria-parasite genome database on the Web",
"UK sets up genome bank to protect rare sheep breeds",
"Dolly's DNA damaged"]

texts=[]
for d in documents:
    # creates an array of tokenized documents
    texts.append(my_tokenizer(d))

# Crea il dizionario per il corpus dei documenti
dictionary = corpora.Dictionary(texts)
bow_corpus = [dictionary.doc2bow(text) for text in texts]

# Creazione del modello TF-IDF
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

# Crea il modello LSI, con 2 argomenti (topics)
lsi_model = models.LsiModel(corpus_tfidf, num_topics=2, id2word=dictionary)

# Crea un indice che facilita il calcolo delle similarità
index = similarities.MatrixSimilarity(lsi_model[corpus_tfidf])
# Stampa l'indice 
for i, similarity in enumerate(index):
    print(f"Documento {i}: {similarity}")

Documento 0: [1.0000001  0.97902197 0.9843578  0.9853968  0.99798703 0.10885489
 0.13524207 0.68249047 0.19551672 0.11114562]
Documento 1: [ 0.97902197  1.          0.9996056   0.9994192   0.9641294  -0.09597263
 -0.0694778   0.5192499  -0.00840719 -0.0936783 ]
Documento 2: [ 0.9843578   0.9996056   1.          0.999982    0.9712031  -0.06798209
 -0.04143595  0.5430449   0.01967745 -0.06568253]
Documento 3: [ 0.9853968   0.9994192   0.999982    1.          0.97261465 -0.06199703
 -0.03544263  0.54807144  0.02567366 -0.05969658]
Documento 4: [0.99798703 0.9641294  0.9712031  0.97261465 1.         0.17167735
 0.19780561 0.7274688  0.25731763 0.1739474 ]
Documento 5: [ 0.10885489 -0.09597263 -0.06798209 -0.06199703  0.17167735  1.
  0.9996466   0.8008437   0.9961556   0.9999973 ]
Documento 6: [ 0.13524207 -0.0694778  -0.04143595 -0.03544263  0.19780561  0.9996466
  1.          0.81648093  0.99813235  0.9997052 ]
Documento 7: [0.68249047 0.5192499  0.5430449  0.54807144 0.7274688  0.800843


**`MatrixSimilarity`** è una classe di Gensim utilizzata per calcolare la similarità tra una serie di documenti in un corpus, rappresentati come vettori numerici. Solitamente, i documenti sono trasformati in una rappresentazione numerica tramite modelli come **TF-IDF** o **LSI** (Latent Semantic Indexing), e poi viene creato un indice che consente calcoli di similarità più rapidi.
`MatrixSimilarity` permette di calcolare la similarità tra i documenti utilizzando un indice che consente di fare ricerche veloci. L'indice è utile per calcolare la similarità coseno tra una query e i documenti nel corpus. 

### Parametri principali di **`MatrixSimilarity`**:
1. **`corpus`**:
   - Descrizione: Una lista di documenti, dove ogni documento è rappresentato come un vettore numerico (di solito ottenuto da un modello come **TF-IDF** o **LSI**).
   - Tipo: `list` (di vettori sparsi).
   
2. **`num_features`**:
   - Descrizione: Il numero di caratteristiche nel vocabolario (tipicamente la dimensione del vocabolario del corpus).
   - Tipo: `int`.
   
3. **`id2word`**:
   - Descrizione: Un dizionario che mappa gli ID numerici ai termini del vocabolario.
   - Tipo: `Dictionary`.
   
### Funzioni principali:
- **`index[query]`**:
   Calcola la similarità tra una query e tutti i documenti nel corpus. La query deve essere rappresentata nello stesso spazio vettoriale del corpus (es. come vettore **TF-IDF**).
   
- **`index.save(filename)`**:
   Salva l'indice su un file, utile per evitare di ricalcolarlo in futuro.

- **`index.load(filename)`**:
   Carica un indice precedentemente salvato.

In [160]:
# Stampa la similarità tra il primo documento e tutti gli altri
print("similarità tra il primo documento e tutti gli altri")
print(list(index)[0])
print("\nsimilarità tra il sesto documento e tutti gli altri") 
print(list(index)[5])

# Stampa il primo argomento del modello LSI
print(lsi_model.show_topic(0))
# Stampa il secondo argomento del modello LSI
print(lsi_model.show_topic(1))

similarità tra il primo documento e tutti gli altri
[1.0000001  0.97902197 0.9843578  0.9853968  0.99798703 0.10885489
 0.13524207 0.68249047 0.19551672 0.11114562]

similarità tra il sesto documento e tutti gli altri
[ 0.10885489 -0.09597263 -0.06798209 -0.06199703  0.17167735  1.
  0.9996466   0.8008437   0.9961556   0.9999973 ]
[('debian', 0.4434854694998973), ('releas', 0.39387184658100105), ('woodi', 0.35142421095730497), ('gentoo', 0.29585355292326376), ('wine', 0.28306005410237095), ('fix', 0.2830600541023709), ('sourc', 0.1741743715775479), ('open', 0.17417437157754778), ('softwar', 0.15920294729105253), ('databas', 0.14610110507158452)]
[('dna', -0.5359729615542277), ('dolli', -0.40426796321228187), ('damag', -0.40273638947754414), ('introduc', -0.18203378118963082), ('news', -0.1820337811896308), ('low-cost', -0.1820337811896308), ('chip', -0.18203378118963076), ('human', -0.18203378118963076), ('genom', -0.17581616785376644), ('total', -0.1756403086703447)]


In [161]:
query_document = my_tokenizer("DNA")

query_bow = dictionary.doc2bow(query_document)
query_lsi = lsi_model[query_bow]  # convert la query nello spazio LSI 

#calvcola la similarità tra query e docs
sims = index[query_lsi]
print(list(enumerate(sims)))

[(0, 0.118368536), (1, -0.08643689), (2, -0.05842559), (3, -0.05243706), (4, 0.18110284), (5, 0.9999541), (6, 0.99985534), (7, 0.8065415), (8, 0.9969488), (9, 0.99997354)]
